In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, explode, to_timestamp, expr, lit 
import boto3
import os
import glob
import shutil

In [2]:
# Start pySpark session
spark = SparkSession.builder \
                    .appName("twitch_emotes_analytics_project") \
                    .master("local[*]") \
                    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/07/06 15:33:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [15]:
# Input from user
channel_name = input("Please Enter the Channel Name: ")
chat_years_input = input("Please Enter the Chat Years (comma-separated): ")
# Convert input to list of years
chat_years = [year.strip() for year in chat_years_input.split(",") if year.strip().isdigit() and len(year.strip()) == 4]
print(chat_years)

Please Enter the Channel Name:  esl_dota2
Please Enter the Chat Years (comma-separated):  2013


['2013']


In [3]:
# S3 setup
bucket = "twitch-emotes-analytics-project"
s3 = boto3.client("s3")

In [18]:
# Local paths
local_path = "/Users/hari14/Desktop/PHD/twitch_emotes_analytics_project/data/processed_silver/"

In [19]:
# order raw s3 files
def order_files(json_files):
    try:
        return sorted(
            json_files,
            key=lambda x: int(os.path.basename(x).replace(".json", "").split("_")[-2])
        )
    except (IndexError, ValueError) as e:
        raise ValueError(f"Filename parsing failed: {e}")

In [21]:
# channel_name = "esl_dota2"
# chat_year = "2016"

In [22]:
s3_key_prefix = f"data/raw_bronze/{channel_name}/{chat_year}/"
print(s3_key_prefix)

data/raw_bronze/esl_dota2/2016/


In [23]:
response = s3.list_objects_v2(Bucket=bucket, Prefix=s3_key_prefix)

In [24]:
contents = response.get("Contents", [])

In [25]:
if not contents:
        print(f"🚫 No files found at S3 path: {s3_key_prefix}")

In [26]:
json_files = [i["Key"] for i in contents]

In [27]:
json_files

['data/raw_bronze/esl_dota2/2016/esl_dota2_2016_000_58748633.json',
 'data/raw_bronze/esl_dota2/2016/esl_dota2_2016_002_58820720.json',
 'data/raw_bronze/esl_dota2/2016/esl_dota2_2016_003_58918503.json',
 'data/raw_bronze/esl_dota2/2016/esl_dota2_2016_004_59285035.json',
 'data/raw_bronze/esl_dota2/2016/esl_dota2_2016_005_59410946.json',
 'data/raw_bronze/esl_dota2/2016/esl_dota2_2016_006_59474511.json',
 'data/raw_bronze/esl_dota2/2016/esl_dota2_2016_007_59482312.json',
 'data/raw_bronze/esl_dota2/2016/esl_dota2_2016_008_59483053.json',
 'data/raw_bronze/esl_dota2/2016/esl_dota2_2016_009_59544034.json',
 'data/raw_bronze/esl_dota2/2016/esl_dota2_2016_010_59544611.json',
 'data/raw_bronze/esl_dota2/2016/esl_dota2_2016_011_59913154.json',
 'data/raw_bronze/esl_dota2/2016/esl_dota2_2016_012_60310974.json',
 'data/raw_bronze/esl_dota2/2016/esl_dota2_2016_013_60334251.json',
 'data/raw_bronze/esl_dota2/2016/esl_dota2_2016_014_60485047.json',
 'data/raw_bronze/esl_dota2/2016/esl_dota2_2016_

In [28]:
sorted_json_files = order_files(json_files)

In [29]:
sorted_json_files

['data/raw_bronze/esl_dota2/2016/esl_dota2_2016_000_58748633.json',
 'data/raw_bronze/esl_dota2/2016/esl_dota2_2016_002_58820720.json',
 'data/raw_bronze/esl_dota2/2016/esl_dota2_2016_003_58918503.json',
 'data/raw_bronze/esl_dota2/2016/esl_dota2_2016_004_59285035.json',
 'data/raw_bronze/esl_dota2/2016/esl_dota2_2016_005_59410946.json',
 'data/raw_bronze/esl_dota2/2016/esl_dota2_2016_006_59474511.json',
 'data/raw_bronze/esl_dota2/2016/esl_dota2_2016_007_59482312.json',
 'data/raw_bronze/esl_dota2/2016/esl_dota2_2016_008_59483053.json',
 'data/raw_bronze/esl_dota2/2016/esl_dota2_2016_009_59544034.json',
 'data/raw_bronze/esl_dota2/2016/esl_dota2_2016_010_59544611.json',
 'data/raw_bronze/esl_dota2/2016/esl_dota2_2016_011_59913154.json',
 'data/raw_bronze/esl_dota2/2016/esl_dota2_2016_012_60310974.json',
 'data/raw_bronze/esl_dota2/2016/esl_dota2_2016_013_60334251.json',
 'data/raw_bronze/esl_dota2/2016/esl_dota2_2016_014_60485047.json',
 'data/raw_bronze/esl_dota2/2016/esl_dota2_2016_

In [30]:
i = 0
key = sorted_json_files[0]
print(i, key)

0 data/raw_bronze/esl_dota2/2016/esl_dota2_2016_000_58748633.json


In [31]:
file_name = os.path.basename(key)
print(file_name)

esl_dota2_2016_000_58748633.json


In [32]:
local_file_path = os.path.join(local_path, file_name)
print(local_path, file_name, local_file_path)

/Users/hari14/Desktop/PHD/twitch_emotes_analytics_project/data/processed_silver/ esl_dota2_2016_000_58748633.json /Users/hari14/Desktop/PHD/twitch_emotes_analytics_project/data/processed_silver/esl_dota2_2016_000_58748633.json


In [34]:
temp_processed_path = os.path.join(local_path, "temp_processed")
print(temp_processed_path)

/Users/hari14/Desktop/PHD/twitch_emotes_analytics_project/data/processed_silver/temp_processed


In [35]:
file_vod_id = file_name.replace(".json", "").split("_")[-1]
print(file_vod_id)

58748633


In [37]:
s3.download_file(Bucket=bucket, Key=key, Filename=local_file_path)

In [38]:
df = spark.read.format("json").option("multiLine", True).load(local_file_path)

ANTLR Tool version 4.9.3 used for code generation does not match the current runtime version 4.7ANTLR Runtime version 4.9.3 used for parser compilation does not match the current runtime version 4.7ANTLR Tool version 4.9.3 used for code generation does not match the current runtime version 4.7ANTLR Runtime version 4.9.3 used for parser compilation does not match the current runtime version 4.7

In [39]:
df.printSchema()

root
 |-- author: struct (nullable = true)
 |    |-- badges: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- clickAction: string (nullable = true)
 |    |    |    |-- clickURL: string (nullable = true)
 |    |    |    |-- icons: array (nullable = true)
 |    |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |    |-- height: long (nullable = true)
 |    |    |    |    |    |-- id: string (nullable = true)
 |    |    |    |    |    |-- url: string (nullable = true)
 |    |    |    |    |    |-- width: long (nullable = true)
 |    |    |    |-- name: string (nullable = true)
 |    |    |    |-- title: string (nullable = true)
 |    |    |    |-- version: long (nullable = true)
 |    |-- colour: string (nullable = true)
 |    |-- display_name: string (nullable = true)
 |    |-- id: string (nullable = true)
 |    |-- name: string (nullable = true)
 |-- emotes: array (nullable = true)
 |    |-- element: struct (cont

In [40]:
df.count()

412

In [41]:
df_filtered = df.filter(col("emotes").isNotNull())

In [42]:
df_filtered.count()

86

In [43]:
df_exploded = df_filtered.withColumn("emote", explode("emotes"))

In [44]:
df_exploded.count()

88

In [45]:
df_exploded.printSchema()

root
 |-- author: struct (nullable = true)
 |    |-- badges: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- clickAction: string (nullable = true)
 |    |    |    |-- clickURL: string (nullable = true)
 |    |    |    |-- icons: array (nullable = true)
 |    |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |    |-- height: long (nullable = true)
 |    |    |    |    |    |-- id: string (nullable = true)
 |    |    |    |    |    |-- url: string (nullable = true)
 |    |    |    |    |    |-- width: long (nullable = true)
 |    |    |    |-- name: string (nullable = true)
 |    |    |    |-- title: string (nullable = true)
 |    |    |    |-- version: long (nullable = true)
 |    |-- colour: string (nullable = true)
 |    |-- display_name: string (nullable = true)
 |    |-- id: string (nullable = true)
 |    |-- name: string (nullable = true)
 |-- emotes: array (nullable = true)
 |    |-- element: struct (cont

In [53]:
# log missing user_name or user_id files
missing_users_files = []
missing_users_df = df_exploded.filter(col("author.id").isNull() | col("author.name").isNull())
missing_users_count = missing_users_df.count() 
if missing_users_count > 0:
    missing_users_files.append(file_vod_id)   

In [54]:
missing_users_files

['58748633']

In [55]:
df_badges = df_exploded.withColumn("i_badge_names", expr("transform(author.badges, x -> x.name)")) \
                                   .withColumn("i_badge_titles", expr("transform(author.badges, x -> x.title)")) \
                                   .withColumn("i_badge_versions", expr("transform(author.badges, x -> x.version)"))

In [56]:
df_badges.printSchema()

root
 |-- author: struct (nullable = true)
 |    |-- badges: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- clickAction: string (nullable = true)
 |    |    |    |-- clickURL: string (nullable = true)
 |    |    |    |-- icons: array (nullable = true)
 |    |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |    |-- height: long (nullable = true)
 |    |    |    |    |    |-- id: string (nullable = true)
 |    |    |    |    |    |-- url: string (nullable = true)
 |    |    |    |    |    |-- width: long (nullable = true)
 |    |    |    |-- name: string (nullable = true)
 |    |    |    |-- title: string (nullable = true)
 |    |    |    |-- version: long (nullable = true)
 |    |-- colour: string (nullable = true)
 |    |-- display_name: string (nullable = true)
 |    |-- id: string (nullable = true)
 |    |-- name: string (nullable = true)
 |-- emotes: array (nullable = true)
 |    |-- element: struct (cont

In [59]:
df_badges.select("i_badge_names", "i_badge_titles", "i_badge_versions").show(3, truncate= False)

+-----------------------+-----------------------+----------------+
|i_badge_names          |i_badge_titles         |i_badge_versions|
+-----------------------+-----------------------+----------------+
|NULL                   |NULL                   |NULL            |
|[moderator, subscriber]|[Moderator, Subscriber]|[1, 0]          |
|NULL                   |NULL                   |NULL            |
+-----------------------+-----------------------+----------------+
only showing top 3 rows



In [64]:
df_exploded.count()

88

In [66]:
# Final select with VOD ID added
df_panel = df_badges.select(
                col("author.id").cast("long").alias("i_user_id"),
                col("author.name").alias("i_user_name"),
                col("author.colour").alias("i_display_color"),
                col("i_badge_names"),
                col("i_badge_titles"),
                col("i_badge_versions").cast("array<string>"),
                lit(None).cast("string").alias("i_user_status"),
                lit(channel_name).alias("j_streamer"),
                col("emote.name").alias("k_emote_name"),
                to_timestamp((col("timestamp") / 1000000).cast("double")).alias("t_timestamp"),
                col("time_text").alias("t_time_text"),
                col("time_in_seconds").cast("long").alias("t_seconds"),
                lit(file_vod_id).alias("vod_id"),
                col("message").alias("chat_message")
            )

In [69]:
df_panel.count()
df_panel.printSchema()

root
 |-- i_user_id: long (nullable = true)
 |-- i_user_name: string (nullable = true)
 |-- i_display_color: string (nullable = true)
 |-- i_badge_names: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- i_badge_titles: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- i_badge_versions: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- i_user_status: string (nullable = true)
 |-- j_streamer: string (nullable = false)
 |-- k_emote_name: string (nullable = true)
 |-- t_timestamp: timestamp (nullable = true)
 |-- t_time_text: string (nullable = true)
 |-- t_seconds: long (nullable = true)
 |-- vod_id: string (nullable = false)
 |-- chat_message: string (nullable = true)



In [70]:
df_panel.show(3)

+---------+-----------+---------------+--------------------+--------------------+----------------+-------------+----------+------------+--------------------+-----------+---------+--------+--------------------+
|i_user_id|i_user_name|i_display_color|       i_badge_names|      i_badge_titles|i_badge_versions|i_user_status|j_streamer|k_emote_name|         t_timestamp|t_time_text|t_seconds|  vod_id|        chat_message|
+---------+-----------+---------------+--------------------+--------------------+----------------+-------------+----------+------------+--------------------+-----------+---------+--------+--------------------+
| 48149174|     pokohh|        #DAA520|                NULL|                NULL|            NULL|         NULL| esl_dota2|    admiralW|2016-04-05 12:43:...|       2:19|      139|58748633|            admiralW|
| 36481935|  esl_dota2|           NULL|[moderator, subsc...|[Moderator, Subsc...|          [1, 0]|         NULL| esl_dota2|          :D|2016-04-05 12:44:...|   

In [71]:
import pandas as pd

In [72]:
# Keep track of files being processed
total_files_per_year = {}
processed_files_per_year = {}
no_emote_files_per_year = {}
missing_users_files_per_year = {}
missing_users_count_per_year = {}

In [74]:
processed_files =[]
no_emotes_files =[]
missing_users_files = []

In [77]:
missing_users_files

['58748633']

In [76]:
# log missing user_name or user_id files
            
missing_users_df = df_exploded.filter(col("author.id").isNull() | col("author.name").isNull())
missing_users_count = missing_users_df.count() 
if missing_users_count > 0:
    missing_users_files.append(file_vod_id)

In [80]:
df_panel.printSchema()

root
 |-- i_user_id: long (nullable = true)
 |-- i_user_name: string (nullable = true)
 |-- i_display_color: string (nullable = true)
 |-- i_badge_names: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- i_badge_titles: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- i_badge_versions: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- i_user_status: string (nullable = true)
 |-- j_streamer: string (nullable = false)
 |-- k_emote_name: string (nullable = true)
 |-- t_timestamp: timestamp (nullable = true)
 |-- t_time_text: string (nullable = true)
 |-- t_seconds: long (nullable = true)
 |-- vod_id: string (nullable = false)
 |-- chat_message: string (nullable = true)



In [80]:
df_badges = df_exploded.withColumn("i_badge_names", expr("transform(author.badges, x -> x.name)")) \
                                   .withColumn("i_badge_titles", expr("transform(author.badges, x -> x.title)")) \
                                   .withColumn("i_badge_versions", expr("transform(author.badges, x -> x.version)"))